## Requirements

In [1]:
!pip install torch
!pip install transformers
!pip install matplotlib
!pip install spacy
!pip install pandas

     |████████████████████████████████| 11.5 MB 4.4 MB/s eta 0:00:01
  Using cached cycler-0.10.0-py2.py3-none-any.whl (6.5 kB)
     |████████████████████████████████| 1.1 MB 50.1 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 49.3 MB/s eta 0:00:01
     |████████████████████████████████| 9.5 MB 5.4 MB/s eta 0:00:01
  Using cached pytz-2021.1-py2.py3-none-any.whl (510 kB)


## BETO

In [1]:
sst_home = '/home/sergio/Escritorio/ProfNER'

In [2]:
!pip install spacy==2.3.5

In [3]:
!python -m spacy download es_core_news_md

import spacy
import re
import es_core_news_md

nlp = es_core_news_md.load()

     |████████████████████████████████| 47.4 MB 8.8 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [11]:
sst_home_train = sst_home + '/profner-data/subtask-2/brat/valid'
print(sst_home_train)
import re
import os
regex = r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))"
texts = []

for file in [file[:-4] for file in os.listdir(sst_home_train) if file.endswith('.txt') and not ' ' in file]:
    file_text = os.path.join(sst_home_train, file + '.txt')
    with open(file_text) as f:
      doc = f.read()
      #print(doc)
      doc = doc.replace('#', '')
      doc = re.sub(regex, '', doc)
      #print(doc)
    
    spacy_text = nlp(doc)
    text = '[CLS] '
    for sent in spacy_text.sents:
        text = text + sent.text + ' [SEP]'
    
    texts.append((text, [token.text for token in spacy_text], file))
            

/home/sergio/Escritorio/ProfNER/profner-data/subtask-2/brat/valid


In [29]:
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

--2021-02-13 15:53:41--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolviendo users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Conectando con users.dcc.uchile.cl (users.dcc.uchile.cl)[200.9.99.211]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 409871727 (391M) [application/x-gzip]
Guardando como: “pytorch_weights.tar.gz”

pytorch_weights.tar 100%[===================>] 390,88M  4,34MB/s    en 5m 33s  

2021-02-13 15:59:16 (1,17 MB/s) - “pytorch_weights.tar.gz” guardado [409871727/409871727]

--2021-02-13 15:59:16--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt
Resolviendo users.dcc.uchile.cl (users.dcc.uchile.cl)... 192.80.24.4, 200.9.99.211
Conectando con users.dcc.uchile.cl (users.dcc.uchile.cl)[192.80.24.4]:443... conectado.
Petición HTTP enviada, esperando respuesta... 200 OK
Longitud: 242120 (236K) [text/plain]
Guardando como: “vocab.txt”

vocab.txt           100%[===============

In [5]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs', do_lower_case=False)
model = BertModel.from_pretrained('/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs/', output_hidden_states = True)
model.eval()

Some weights of BertModel were not initialized from the model checkpoint at /home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs/ and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(31002, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [12]:
def getVectors(tokens, tokens_vectors, spacy_tokens, BETO_DIM = 1536):
    beto_2_spacy = {}
    beto_enum = {}
    special_tokens = ['[CLS]', '[SEP]', '[UNK]']

    token2txt = ''
    token2vect = []
    for token, vector in zip(tokens, tokens_vectors):
        if token in special_tokens:
            pass
        else:
            if not token.startswith('#'):
                if token2txt != '':
                    mean = torch.mean(torch.stack(token2vect), dim = 0)
                    mean = mean.cpu().detach().numpy()
                    idx = beto_enum.get(token2txt, -1) + 1
                    beto_2_spacy[(token2txt, idx)] = mean
                token2txt = token
                token2vect = [vector]
            else:
                token2txt = token2txt + token.replace('#','')
                token2vect.append(vector)
    if token2txt != '':
        mean = torch.mean(torch.stack(token2vect), dim = 0)
        mean = mean.cpu().detach().numpy()
        idx = beto_enum.get(token2txt, -1) + 1
        beto_2_spacy[(token2txt, idx)] = mean
    
    tensor_enum = {}
    tweet_data = []
    for token in spacy_tokens:
        idx = tensor_enum.get(token, -1) + 1
        if (token, idx) in beto_2_spacy:
            tweet_data.append(beto_2_spacy[(token,idx)])
        else:
            tweet_data.append(np.zeros(BETO_DIM))
        
    return np.array(tweet_data)
            
        

In [13]:
data = {}
import numpy as np
# Now test it
for elements in texts:
    text = elements[0]
    spacy_tokens = elements[1]
    file_name = elements[2]
    
    tokens = tokenizer.tokenize(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)
    tokens_tensor = torch.tensor([indexed_tokens])

    segments_ids = [1] * len(tokens)
    segments_tensors = torch.tensor([segments_ids])

    with torch.no_grad():

        outputs = model(tokens_tensor, segments_tensors)

        # Evaluating the model will return a different number of objects based on 
        # how it's  configured in the `from_pretrained` call earlier. In this case, 
        # becase we set `output_hidden_states = True`, the third item will be the 
        # hidden states from all layers. See the documentation for more details:
        # https://huggingface.co/transformers/model_doc/bert.html#bertmodel
        #print(outputs)

        hidden_states = outputs[2]

    # Concatenate the tensors for all layers. We use `stack` here to
    # create a new dimension in the tensor.
    token_embeddings = torch.stack(hidden_states, dim=0)
    # Remove dimension 1, the "batches".
    token_embeddings = torch.squeeze(token_embeddings, dim=1)
    # Swap dimensions 0 and 1.
    token_embeddings = token_embeddings.permute(1,0,2)
    
    # Stores the token vectors, with shape [22 x 768]
    token_vecs_sum = []

    # `token_embeddings` is a [22 x 12 x 768] tensor.

    # For each token in the sentence...
    for token in token_embeddings:

        # `token` is a [12 x 768] tensor

        # Sum the vectors from the last four layers.
        sum_vec_1 = torch.mean(token[-2:], dim=0)
        sum_vec_2 = torch.mean(token[-4:-2], dim=0)
        sum_vec = torch.cat((sum_vec_1, sum_vec_2))
        

        # Use `sum_vec` to represent `token`.
        token_vecs_sum.append(sum_vec)
    #print ('Shape is: %d x %d' % (len(token_vecs_sum), len(token_vecs_sum[0])))
    data[file_name] = getVectors(tokens, token_vecs_sum, spacy_tokens)

In [11]:
!pip install pickle5

  Using cached pickle5-0.0.11.tar.gz (132 kB)
  Created wheel for pickle5: filename=pickle5-0.0.11-cp36-cp36m-linux_x86_64.whl size=247167 sha256=64ce560b25bdf24715a461cb168319b43f52cdfcfacb4d43d5c3714b97ea74b0
  Stored in directory: /home/sergio/.cache/pip/wheels/f9/b7/be/bf9768ab0daa28fa4b386f3ad1bac5dd4d9c349c60e83b24e3
Successfully built pickle5


In [14]:
import pickle5 as pickle
with open('/home/sergio/Escritorio/ProfNER/saved_data/bert_valid.pickle', 'wb') as file:
    pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

## BETO fine-tuning to ProfNER

In [1]:
sst_home = '/home/sergio/Escritorio/ProfNER'

In [2]:
!python -m spacy download es_core_news_md

import spacy
import re
import es_core_news_md

nlp = es_core_news_md.load()

     |████████████████████████████████| 47.4 MB 9.9 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_md')


In [3]:
############# BIOES NOTATION #####################
BEGIN = 'B'
INSIDE = 'I'
OUTSIDE = 'O'
END = 'E'
SINGLE = 'S'

def getDictEntities(file_ann, ent_classes = ['PROFESION', 'SITUACION_LABORAL']):
  entities = {}
  with open(file_ann) as anns:
    for ann in anns:
      if ann.split('\t')[1].split(' ')[0] in ent_classes:
          ent = ann[:-1].split('\t')[2]
          #print(ent)
          #ent = [token for token in nlp(ent) if not token.is_stop]
          ent = nlp(ent)
          start = int(ann[:-1].split('\t')[1].split(' ')[1])
          end = int(ann[:-1].split('\t')[1].split(' ')[2])
          if (len(ent) == 1):
            entities[(start, end)] = SINGLE + '_' + ann.split('\t')[1].split(' ')[0]
          else:
            entities[(start, start + len(ent[0].text))] = BEGIN + '_' + ann.split('\t')[1].split(' ')[0]
            entities[(end - len(ent[-1].text)), end] = END + '_' + ann.split('\t')[1].split(' ')[0]
            for i in range(len(ent) - 2):
              spaces = (ent[i + 1].idx) - (ent[i].idx + len(ent[i].text))
              start = start + len(ent[i].text) + spaces
              entities[(start, start + len(ent[i + 1].text))] = INSIDE + '_' + ann.split('\t')[1].split(' ')[0]
            
  return entities


In [4]:
getDictEntities('/home/sergio/Escritorio/ProfNER/profner-data/subtask-2/brat/valid/1242407018465579008.ann')

{(31, 41): 'S_PROFESION'}

In [5]:
import os
def getProccessText(sst_home):
    sentences = []
    labels = []
    for file in [file[:-4] for file in os.listdir(sst_home) if file.endswith('.txt') and not ' ' in file]:
        file_text = os.path.join(sst_home, file + '.txt')
        file_ann = os.path.join(sst_home, file + '.ann')
        _entities = getDictEntities(file_ann)
        with open(file_text) as f:
          text = f.read()
          spacy_text = nlp(text)
          for sent in spacy_text.sents:
            sentence = []
            sent_labels = []
            for token in sent:
                sentence.append(token.text.replace('#',''))
                entity = _entities.get((token.idx, token.idx + len(token.text)), 'O')
                sent_labels.append(entity)
            sentences.append(sentence)
            labels.append(sent_labels)
    
    return sentences, labels


In [6]:
sst_home_train = sst_home + '/profner-data/subtask-2/brat/train'
sentences, labels = getProccessText(sst_home_train)

sst_home_test = sst_home + '/profner-data/subtask-2/brat/valid'
sentences_valid, labels_valid = getProccessText(sst_home_test)

In [7]:
tag_values = ['B_SITUACION_LABORAL', 'I_SITUACION_LABORAL', 'E_SITUACION_LABORAL', 'S_SITUACION_LABORAL' ,
      'B_PROFESION', 'I_PROFESION', 'E_PROFESION', 'S_PROFESION',
      'O']
tag_values.append("PAD")
tag2idx = {t: i for i, t in enumerate(tag_values)}


In [8]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertConfig

from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

torch.__version__

'1.7.1'

In [9]:
MAX_LEN = 75
bs = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [10]:
torch.cuda.get_device_name(0)

'GeForce GTX 1050 Ti'

In [11]:
!pip3 install IProgress
import ipywidgets
import IProgress

In [11]:
import torch
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', do_lower_case=False)

In [12]:
def tokenize_and_preserve_labels(sentence, text_labels):
    tokenized_sentence = []
    labels = []

    for word, label in zip(sentence, text_labels):

        # Tokenize the word and count # of subwords the word is broken into
        tokenized_word = tokenizer.tokenize(word)
        n_subwords = len(tokenized_word)

        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

        # Add the same label to the new list of labels `n_subwords` times
        labels.extend([label] * n_subwords)

    return tokenized_sentence, labels

In [13]:
tokenized_texts_and_labels = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences, labels)
]

tokenized_texts_and_labels_test = [
    tokenize_and_preserve_labels(sent, labs)
    for sent, labs in zip(sentences_valid, labels_valid)
]

In [14]:
tokenized_texts = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels]
labels = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels]

tokenized_texts_test = [token_label_pair[0] for token_label_pair in tokenized_texts_and_labels_test]
labels_test = [token_label_pair[1] for token_label_pair in tokenized_texts_and_labels_test]

In [15]:
input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

input_ids_test = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts_test],
                          maxlen=MAX_LEN, dtype="long", value=0.0,
                          truncating="post", padding="post")

In [16]:
tags = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

tags_test = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels_test],
                     maxlen=MAX_LEN, value=tag2idx["PAD"], padding="post",
                     dtype="long", truncating="post")

In [17]:
attention_masks = [[float(i != 0.0) for i in ii] for ii in input_ids]
attention_masks_test = [[float(i != 0.0) for i in ii] for ii in input_ids_test]

In [18]:
tr_inputs, tr_tags, tr_masks = input_ids, tags, attention_masks
val_inputs, val_tags, val_masks = input_ids_test, tags_test, attention_masks_test

'''tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,
                                                            random_state=2018, test_size=0.1)
tr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)'''

'tr_inputs, val_inputs, tr_tags, val_tags = train_test_split(input_ids, tags,\n                                                            random_state=2018, test_size=0.1)\ntr_masks, val_masks, _, _ = train_test_split(attention_masks, input_ids,\n                                             random_state=2018, test_size=0.1)'

In [19]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)

In [20]:
train_data = TensorDataset(tr_inputs, tr_masks, tr_tags)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=bs)

valid_data = TensorDataset(val_inputs, val_masks, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=bs)

In [21]:
import transformers
from transformers import BertForTokenClassification, AdamW

transformers.__version__

'4.3.2'

In [22]:
model = BertForTokenClassification.from_pretrained(
    '/home/sergio/Escritorio/ProfNER/pytorch/',
    num_labels=len(tag2idx),
    output_attentions = False,
    output_hidden_states = False
)


Some weights of the model checkpoint at /home/sergio/Escritorio/ProfNER/pytorch/ were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model ch

In [23]:
model.cuda();

In [25]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters())
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]

optimizer = AdamW(
    optimizer_grouped_parameters,
    lr=3e-5,
    eps=1e-8
)

In [26]:
from transformers import get_linear_schedule_with_warmup

epochs = 4
max_grad_norm = 1.0

# Total number of training steps is number of batches * number of epochs.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


In [27]:
from seqeval.metrics import f1_score, accuracy_score

In [28]:

import numpy as np
from tqdm import trange
## Store the average loss after each epoch so we can plot them.
loss_values, validation_loss_values = [], []

for _ in trange(epochs, desc="Epoch"):
    # ========================================
    #               Training
    # ========================================
    # Perform one full pass over the training set.

    # Put the model into training mode.
    model.train()
    # Reset the total loss for this epoch.
    total_loss = 0

    # Training loop
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # Always clear any previously calculated gradients before performing a backward pass.
        model.zero_grad()
        # forward pass
        # This will return the loss (rather than the model output)
        # because we have provided the `labels`.
        outputs = model(b_input_ids, token_type_ids=None,
                        attention_mask=b_input_mask, labels=b_labels)
        # get the loss
        loss = outputs[0]
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # track train loss
        total_loss += loss.item()
        # Clip the norm of the gradient
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over the training data.
    avg_train_loss = total_loss / len(train_dataloader)
    print("Average train loss: {}".format(avg_train_loss))

    # Store the loss value for plotting the learning curve.
    loss_values.append(avg_train_loss)


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    # Put the model into evaluation mode
    model.eval()
    # Reset the validation loss for this epoch.
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in valid_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        # Telling the model not to compute or store gradients,
        # saving memory and speeding up validation
        with torch.no_grad():
            # Forward pass, calculate logit predictions.
            # This will return the logits rather than the loss because we have not provided labels.
            outputs = model(b_input_ids, token_type_ids=None,
                            attention_mask=b_input_mask, labels=b_labels)
        # Move logits and labels to CPU
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences.
        eval_loss += outputs[0].mean().item()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.extend(label_ids)

    eval_loss = eval_loss / len(valid_dataloader)
    validation_loss_values.append(eval_loss)
    print("Validation loss: {}".format(eval_loss))
    pred_tags = [tag_values[p_i] for p, l in zip(predictions, true_labels)
                                 for p_i, l_i in zip(p, l) if tag_values[l_i] != "PAD"]
    valid_tags = [tag_values[l_i] for l in true_labels
                                  for l_i in l if tag_values[l_i] != "PAD"]
    print("Validation Accuracy: {}".format(accuracy_score(pred_tags, valid_tags)))
    #print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    print()

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Average train loss: 0.04292192439080926


Epoch:  25%|██▌       | 1/4 [07:55<23:46, 475.54s/it]

Validation loss: 0.021197308957714517
Validation Accuracy: 0.9939513640291322

Average train loss: 0.012423428417517245


Epoch:  50%|█████     | 2/4 [16:04<16:07, 483.57s/it]

Validation loss: 0.020311886659316006
Validation Accuracy: 0.9948066376285115

Average train loss: 0.005480610090717899


Epoch:  75%|███████▌  | 3/4 [24:17<08:07, 487.57s/it]

Validation loss: 0.021707191045213653
Validation Accuracy: 0.9954767489022519

Average train loss: 0.002368010606933954


Epoch: 100%|██████████| 4/4 [32:29<00:00, 487.48s/it]

Validation loss: 0.02382261357504069
Validation Accuracy: 0.9953973936198353



In [29]:
model.bert.save_pretrained('/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs')

In [30]:
tokenizer.save_pretrained('/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs')

('/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs/tokenizer_config.json',
 '/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs/special_tokens_map.json',
 '/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs/vocab.txt',
 '/home/sergio/Escritorio/ProfNER/fine-tuned-bert_3_epochs/added_tokens.json')

## BETO Cosine Similarity Embeddings

### Getting the Entity Vectors